# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# load data from database
from sqlalchemy import create_engine

engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_response', engine)

In [4]:
df.shape

(26216, 40)

In [5]:
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [6]:
X = df.message
y = df.iloc[:,4:]

In [7]:
X.head(3)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
Name: message, dtype: object

In [8]:
y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# additional data cleaning - source https://knowledge.udacity.com/questions/764236
y.related.replace(2,1,inplace=True)

### 2. Write a tokenization function to process your text data

In [10]:
import re
import nltk
nltk.download(['punkt','wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def tokenize(text):
    
    # remove punctuation, lowercase
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize
    tokens = word_tokenize(text)
    
    # lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer = tokenize)),
    ('classifier', OneVsRestClassifier(LogisticRegression(max_iter = 200))),
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = True, random_state=42)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
pipeline.fit(X_train, y_train);

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))


In [14]:
from sklearn.metrics import classification_report

y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)

print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values));


                        precision    recall  f1-score   support

               related       0.83      0.95      0.89      5992
               request       0.83      0.56      0.67      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.76      0.67      0.71      3219
          medical_help       0.67      0.14      0.23       638
      medical_products       0.86      0.15      0.25       418
     search_and_rescue       1.00      0.04      0.08       192
              security       0.00      0.00      0.00       144
              military       0.55      0.07      0.12       245
           child_alone       0.00      0.00      0.00         0
                 water       0.79      0.44      0.56       500
                  food       0.86      0.56      0.68       878
               shelter       0.82      0.43      0.57       705
              clothing       0.71      0.09      0.16       115
                 money       0.69      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [15]:
pipeline.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7ff78fd29b70>, use_idf=True,
           vocabulary=None)),
  ('classifier',
   OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=200, multi_class='ovr', n_jobs=1,
             penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
             verbose=0, warm_start=False),
             n_jobs=1))],
 'tfidf': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<cla

In [20]:
from sklearn.model_selection import GridSearchCV

parameters_grid = {'classifier__estimator__class_weight': [None,'balanced'],
              'classifier__estimator__penalty': ['l1', 'l2']}



cv = GridSearchCV(pipeline, param_grid=parameters_grid, scoring='f1_micro')

In [21]:
cv.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training examples.
  str(classes[c]))
/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 9 is present in all training example

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'classifier__estimator__class_weight': [None, 'balanced'], 'classifier__estimator__penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=0)

In [17]:
cv.best_params_

{'classifier__estimator__class_weight': None,
 'classifier__estimator__penalty': 'l1'}

In [18]:
cv.best_score_

0.67888184827279618

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
tuned_model = cv.best_estimator_


y_pred_train = tuned_model.predict(X_train)
y_pred_test = tuned_model.predict(X_test)

print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values));


                        precision    recall  f1-score   support

               related       0.84      0.94      0.89      5992
               request       0.81      0.58      0.67      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.77      0.67      0.71      3219
          medical_help       0.60      0.23      0.33       638
      medical_products       0.70      0.26      0.38       418
     search_and_rescue       0.76      0.15      0.25       192
              security       0.00      0.00      0.00       144
              military       0.58      0.25      0.35       245
           child_alone       0.00      0.00      0.00         0
                 water       0.77      0.55      0.64       500
                  food       0.84      0.68      0.75       878
               shelter       0.77      0.56      0.65       705
              clothing       0.78      0.34      0.47       115
                 money       0.57      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [20]:
import pickle
pickle.dump(tuned_model, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.